## Red-teaming postprocessing
The goal here is to turn the Argilla labels labelled by Matyas into Score objects and merge them with the UNECE sprint labels. Important to note, that the policy-violation and UNECE sprint questions are opposing ("does the answer violate the guideline" and "does the answer follow the guideline" respectively) hence flipping the scores is required. 

In [1]:
import os
import sys
import json
import argilla as rg
import pandas as pd

from pathlib import Path

sys.path.append(Path(".").absolute().parent.parent.as_posix())

from src.evaluation.evaluator import Score

In [2]:
rg.init(
    api_url="https://argilla.labs.climatepolicyradar.org/",
    api_key=os.getenv("ARGILLA_KEY"),
)

/Users/matyasjuhasz/git/rag-labs/.venv/lib/python3.11/site-packages/argilla/client/client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(
/Users/matyasjuhasz/git/rag-labs/.venv/lib/python3.11/site-packages/argilla/client/client.py:195: UserWarning: You're connecting to Argilla Server 1.26.1 using a different client version (1.29.0).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [3]:
argilla_datasets = []

batch_name = "policy-violation-002"
user = "matyas"

_argilla_ds = rg.FeedbackDataset.from_argilla(
    name=batch_name, workspace=user
)
_ds = _argilla_ds.format_as("datasets")

_ds = _ds.map(
    lambda x: {
        "user": user,
        "batch_name": batch_name,
    }
)

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

In [4]:
answer_to_score = {
    "YES": 1.0,
    "NO": 0.0
}

scores = []

for _, row in _ds.to_pandas().iterrows():
    uuid = json.loads(row["metadata"])["uuid"]

    if row["policy-violation"]:
        score = answer_to_score[row["policy-violation"][0]["value"]]
    
        # We're only using this dataset for the positives on the violation
        if score == 1.0:
            scores.append(
                Score(
                    gen_uuid=uuid,
                    score=score,
                    type="human",
                    name="cpr-policy-violation"
                )
            )

/var/folders/c1/skfn6y1s1432yhzx3z_2zrph0000gn/T/ipykernel_71931/2949695185.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if row["policy-violation"]:


In [5]:
len(scores)

156

In [ ]:
with open("../../data/eval/cpr_generation_policy_red_teaming_scores.jsonl", "w") as file:
    for score in scores:
        file.write(score.model_dump_json() + "\n")